#importing libraries

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

#Create Flag Parameter

In [0]:
dbutils.widgets.text('incremental_flag', '0')

In [0]:
incremental_flag = dbutils.widgets.get('incremental_flag')
print(incremental_flag)

0


# Creating Dimension Model

###Fetch Relative columns

In [0]:

df_src = spark.sql('''select distinct(Dealer_ID) as Dealer_ID, DealerName
from parquet.`abfss://silver@carsbaggiodatalake.dfs.core.windows.net/carsales`
''')

###Dim_model Sink - Initial and Incremental( Bring Schema if Model not exists)

In [0]:
if spark.catalog.tableExists('cars_catalog.gold.dim_dealer') :

      df_sink = spark.sql('''SELECT dim_dealer_key, Dealer_ID, DealerName
      FROM cars_catalog.gold.dim_dealer
     ''')

else:
       df_sink = spark.sql('''SELECT 1 as dim_dealer_key, Dealer_ID, DealerName 
       FROM parquet.`abfss://silver@carsbaggiodatalake.dfs.core.windows.net/carsales`
       where 1=0''')


### Filtering new records and Old records

In [0]:
df_filter = df_src.join(df_sink, df_src['Dealer_ID'] == df_sink['Dealer_ID'], 'left').select(df_src['Dealer_ID'], df_src['DealerName'], df_sink['dim_dealer_key'])

###df_filer_old

In [0]:
df_filter_old = df_filter.filter(col('dim_dealer_key').isNotNull())

###df_filer_new

In [0]:
df_filter_new = df_filter.filter(col('dim_dealer_key').isNull()).select(df_src['Dealer_ID'], df_src['DealerName'])

In [0]:
df_filter_new.display()

Dealer_ID,DealerName
DLR0058,Fiat do Brasil Motors
DLR0107,Land Rover Motors
DLR0129,Mia Motors
DLR0111,Lotus Motors
DLR0085,Humber Motors
DLR0001,AC Cars Motors
DLR0218,Lagonda Motors
DLR0082,Honda Motors
DLR0063,Ford do Brasil Motors
DLR0193,Tazzari Motors


###Create Surrogate Key

###Fetch the max Surrogate Key from existing table

In [0]:
if (incremental_flag =='0'):
    max_value = 0
else:
    max_value_df = spark.sql("SELECT MAX(dim_dealer_key) FROM cars_catalog.gold.dim_dealer")
    max_value = max_value_df.collect()[0][0]

###Create Surrogate Key column and ADD the max surrogate key

In [0]:
df_filter_new = df_filter_new.withColumn("dim_dealer_key", max_value + monotonically_increasing_id())

###Create Final DF - df_filter_old + df-filter-new

In [0]:
df_final = df_filter_new.union(df_filter_old)

#SCD TYPE-1(UPSERT)

In [0]:
from delta.tables import DeltaTable

In [0]:
#Incremental RUN
if spark.catalog.tableExists('cars_catalog.gold.dim_dealer'):
    delta_tbl = DeltaTable.forPath(spark, "abfss://gold@carsbaggiodatalake.dfs.core.windows.net/dim_dealer")
    delta_tbl.alias("trg").merge(df_final.alias("src"), "trg.dim_dealer_key = src.dim_dealer_key")\
                          .whenMatchedUpdateAll()\
                          .whenNotMatchedInsertAll()\
                          .execute()
                         
                          
    
#Initial RUN
else:
    df_final.write.format("delta")\
            .mode("overwrite")\
            .option("path", "abfss://gold@carsbaggiodatalake.dfs.core.windows.net/dim_dealer")\
            .saveAsTable("cars_catalog.gold.dim_dealer")

In [0]:
%sql
select * from cars_catalog.gold.dim_dealer

Dealer_ID,DealerName,dim_dealer_key
DLR0058,Fiat do Brasil Motors,0
DLR0107,Land Rover Motors,1
DLR0129,Mia Motors,2
DLR0111,Lotus Motors,3
DLR0085,Humber Motors,4
DLR0001,AC Cars Motors,5
DLR0218,Lagonda Motors,6
DLR0082,Honda Motors,7
DLR0063,Ford do Brasil Motors,8
DLR0193,Tazzari Motors,9
